# Systematic Risk

Systematic risk refers to the likelihood of financial instability that can lead to widespread failures within the financial system. Unlike other risks, systematic risk cannot be mitigated through diversification, only through hedging. In an economic downturn, the bankruptcy of one firm cannot be easily absorbed by stronger competitors. Given its complex nature, there are multiple definitions and models for systematic risk. Monitoring this risk is crucial as it helps systematically important financial institutions and policymakers calibrate or limit risk exposure.

### Systematic Risk Review

Value-At-Risk is the most a financial firm can loss over a period of time with some confidence $1-\alpha$. Formally VaR of $R$ is defined as follows


$$

VaR_\alpha(R) = -\inf\{x:\mathbb{P}[R \le r] \ge \alpha \}
= -\sup\{x:\mathbb{P}[R \le r] <\alpha \}

$$

VaR is a basic model with several well-known limitations, including a lack of sub-additivity and failure to consider tail dependency. EIn contrast, Expected Shortfall (ES) improves upon VaR as it is a coherent risk measure that satisfies monotonicity, sub-additivity, positive homogeneity, and translation invariance. Expected shortfall is the expected loss conditional on returns being less that VaR. Expected shortfall measures the average loss in scenarios where losses exceed the VaR. In a market with $n$ financial institutions the overall return $R$ can be decomposed into contributions $w_i$ from individual returns $r_i$, that is $R=\sum_{i=1}^{n} w_ir_i$ 

$$

\begin{split}
	ES_\alpha &= -\mathbb{E}[R|R\le -Var] \\
&= -\sum_{i=1}^{n} w_i\mathbb{E}[r_i|R \le -VaR]\\
\end{split}
$$

It follows from above that the Marginal Expected Shortfall(MES), that is the risk contribution of a single institution $i$ to the total risk can be expressed as follows.

$$
MES_i = \frac{\partial ES_a}{\partial w_i} = - \mathbb{E}[r_i|R \le -VaR]

$$

A drawback of expected shortfall is that it cannot be used proactively since it relies on the actual occurrence of the risk. We extend this definition to account for the market value of an institution $M_t$, $ M_t=W_t+D_t$ where ($W_t$) is the working capital and $D_t$ is the debt of said institution at time $t$. The expected capital shortfall is the amount lost during a crisis. The debt remains unchanged as it cannot be renegotiated during the crisis.

$$
\begin{split}
ECS_t&=\mathbb{E}_t[k(D_{t+h}+W_{t+h})-W_{t+h}|R_{\Delta t}\lt C]\\
&= kD_t -(1-k)\mathbb{E}[W_{t+h}|R_{\Delta t}\lt C] \\
&= kD_t -(1-k)W_t\mathbb{E}_t[R_{i,t+h}|R_{m,t+h}\lt C] \\
&= kD_t -(1-k)W_t(1-LRMES) 
\end{split}

$$

The Long Run Marginal Expected Shortfall (LRMES) is an extension of MES over horizon $h$, is defined as $LRMES= - \mathbb{E}_t[R_{i,t+h}|R_{m,t+h}\lt C]$. When the capital shortfall is negative, it indicates that the firm is functioning properly. However, a positive capital shortfall suggests that the firm is at risk. Factors contributing to capital shortfall include the firm's size, leverage, and risk exposure.